# Libraries Import

In [1]:
import pandas as pd
import numpy as np
import os.path
import recsys as rs
import notipy
import TopSimilarRecommender as TSR
import random

# Datasets

In [2]:
train = pd.read_csv('Data/train_final.csv','\t')
tr_info = pd.read_csv('Data/tracks_final.csv','\t')
pl_info = pd.read_csv('Data/playlists_final.csv','\t')
tgt_pl = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tr = pd.read_csv('Data/target_tracks.csv','\t')

# Data Visualization

In [3]:
train.head()

,playlist_id,track_id
0,3271849,2801526
1,5616275,727878
2,11267488,2805283
3,10103900,1515105
4,3836898,2945623


In [4]:
tr_info.head()

,track_id,artist_id,duration,playcount,album,tags
0,2972914,144,224000,49.0,[7],"[54087, 1757, 1718, 116712, 189631]"
1,2750239,246,157000,1.0,[8],"[189631, 3424, 177424, 46208, 205245]"
2,1550729,144,217000,554.0,[9],"[54087, 109806, 46869, 183258, 54337]"
3,2169950,144,207000,200.0,[9],"[54087, 70618, 207003, 109806, 116712]"
4,1903709,144,198000,5.0,[None],"[54087, 81223, 116712, 215342, 71028]"


In [5]:
pl_info.head()

,created_at,playlist_id,title,numtracks,duration,owner
0,1216545588,644838,[12727],27,6522,41504
1,1249326867,7577564,[],9,2650,41504
2,1257766688,3120683,[183],16,3645,44542
3,1248079275,4278112,"[12389, 18698, 18925, 11695, 7117]",15,4151,44542
4,1175201268,8656823,"[12809, 2095, 13257, 12671, 20426, 14448, 18698]",84,18414,44542


In [6]:
tgt_pl.head()

,playlist_id
0,10024884
1,10624787
2,4891851
3,4267369
4,65078


In [7]:
tgt_tr.head()

,track_id
0,1316175
1,3885714
2,3091270
3,226759
4,230596


# Division of the Data in Trainset and Testset

From the target playlists, only the ones with more than 10 elements are kept.

In [8]:
big_target_playlist = [i for i in np.unique(train['playlist_id'].values) if train[train['playlist_id'] == i].shape[0] >= 10]

From the playlists in big_target_playlists, 10 are randomly picked.

In [9]:
test_pl = pd.DataFrame({"playlist_id": list(map(lambda x: random.choice(big_target_playlist), range(10)))})
test_pl

,playlist_id
0,11202811
1,5712207
2,10187858
3,64097
4,3529739
5,10153542
6,11635954
7,10964715
8,5310372
9,7304057


5 elements are removed from each playlist and save in tracks_removed. 
# TO DO: create an empty array and save indexes there, then remove the values

In [31]:
tracks_in_test_pl = pd.merge(train, test_pl, how='inner', on='playlist_id')
tracks_removed= pd.DataFrame(columns=['playlist_id', 'track_id'])
indexes_to_remove= np.array([])
print(tracks_in_test_pl.shape)
for i in np.unique(tracks_in_test_pl['playlist_id'].values):
    #randomly selects 5 elements from playlist i and save indexes
    tmp = (tracks_in_test_pl.where(tracks_in_test_pl['playlist_id']==i).dropna().sample(5)).index
    #insert the items selected in tracks_removed dataFrame
    tracks_removed= tracks_removed.append(tracks_in_test_pl.take(tmp)) 
    #remove tracks from the original dataframe
    indexes_to_remove=np.append(indexes_to_remove, values=tmp)
tracks_in_test_pl.drop(indexes_to_remove, inplace='true')
printtracks_removed.shape
print(tracks_in_test_pl.shape)

,playlist_id,track_id
0,11202811,2754398
1,11202811,3016510
2,11202811,1508676
3,11202811,705384
4,11202811,2823733
5,11202811,2940090
6,11202811,2901701
7,11202811,3231943
8,11202811,2009725
9,11202811,1887605


NameError: name 'fit' is not defined

In [295]:
fit_dict = {'tracks_info' : tr_info,
            'attributes' : ['artist', 'album', 'tags'],
            'tgt_tracks' : tgt_tr,
            'n_min_attr' : 2,
            'measure' : 'dot',
            'shrinkage' : 0,
            'n_el_sim' : 20}

recommend_dict = {'tgt_playlists' : tgt_pl,
                  'train_playlists_tracks_pairs' : train, 
                  'normalize' : False}

In [297]:
rec = TSR()
rec.fit(**fit_dict)
notipy.notify('Model fitted!')
print('Model fitted!')

TypeError: 'module' object is not callable

In [ ]:
recommendetions = rec.recommend(**recommend_dict)

In [ ]:
map_eval = rs.evaluate(recommendetions, test, 'MAP')

In [ ]:
tojson('path', [fit_dict, recommend_dict, map_eval])